Projeto Dados Abertos BNDES

Download bases do BNDES 

In [ ]:
!pip install jupyterlab==3.0.16
!pip install matplotlib==3.4.2
!pip install numpy==1.21.1
!pip install pandas==1.3.1
!pip install requests==2.26.0
!pip install seaborn==0.11.1
# woraround caso o venv seja criado com pip no conda
!pip install pywin32==225

In [ ]:
import requests
from pathlib import Path

import pandas as pd

In [ ]:
config = {
    "endpoint": "https://db-nosql.documents.azure.com:443/",
    "database" : "db-nosql",
    "container" : "meubndes",
    "masterkey": "bQPdXS0y1GLPLeCh2neFzdNX9bd6RKq1J8pTjzsRQG8Tf7DV0xyOe24lwtRTKwF6NbXHV9JRBACIOvfHPHOOHw==",
    "readkey" : "Q9ikAx2k6puQIrVHHN1xosrfAF0amUqTmBC724wJdNxTP3JGNKnQMpixOOkHhgWiVyCMi4xZup7rMeetig1eWg=="
}

In [ ]:
#O URL está sempre no topo da página de cada recurso.
def download_bndes(url_csv):
    Path('dados').mkdir(exist_ok=True)
    arq = Path('dados') / url_csv.split('/')[-1]

    content = None
    if not arq.exists():
        # baixa arquivo
        dados = requests.get(url_csv)
        dados.raise_for_status()
        content = dados.content
        with arq.open('wb') as f:
            f.write(content)
    else:
        print(f"Arquivo já existe, para forçar download apague o arquivo {arq} e tente de novo")
    return arq

In [1]:
urls = {
       "operacoes-automaticas": "https://dadosabertos.bndes.gov.br/dataset/0f335c85-92a8-4343-9423-f073fb40774e/resource/9534f677-9525-4bf8-a3aa-fd5d3e152a93/download/operacoes-financiamento-operacoes-indiretas-automaticas.csv",
       "operacoes-nao-automaticas": "https://dadosabertos.bndes.gov.br/dataset/0f335c85-92a8-4343-9423-f073fb40774e/resource/332d446e-d340-46ef-af64-ee6f36e7bd50/download/operacoes-financiamento-operacoes-nao-automaticas.csv"
}

In [7]:
arquivo = 'operacoes-automaticas'
url_arquivo = urls['operacoes-automaticas']

In [ ]:
arq_dados = download_bndes(url_arquivo)

In [ ]:
df = pd.read_csv(arq_dados, sep=';', decimal=',', encoding='UTF-8') #parse_dates=['nome_col']
df.head()

,cliente,cnpj,descricao_do_projeto,uf,municipio,municipio_codigo,numero_do_contrato,data_da_contratacao,valor_contratado_reais,valor_desembolsado_reais,fonte_de_recurso_desembolsos,custo_financeiro,juros,prazo_carencia_meses,prazo_amortizacao_meses,modalidade_de_apoio,forma_de_apoio,produto,instrumento_financeiro,inovacao,area_operacional,setor_cnae,subsetor_cnae_agrupado,subsetor_cnae_codigo,subsetor_cnae_nome,setor_bndes,subsetor_bndes,porte_do_cliente,natureza_do_cliente,instituicao_financeira_credenciada,cnpj_da_instituicao_financeira_credenciada,tipo_de_garantia,tipo_de_excepcionalidade,situacao_do_contrato
0,MUNICIPIO DE RIBEIRAO PRETO,56.024.581/0001-56,PROGRAMA DE MODERNIZACAO DA ADMINISTRACAO TRIB...,SP,RIBEIRAO PRETO,3543402,1247021,2002-01-02,9090000.00,9007445.10,RECURSOS VINCULADOS - PIS/PASEP,TJLP,2.5,24,72,REEMBOLSÁVEL,DIRETA,BNDES FINEM,BNDES PMAT,NÃO,AREA DE GESTAO PUBLICA E SOCIOAMBIENTAL,COMERCIO E SERVICOS,ADMINISTRAÇÃO PÚBLICA,O8411600,ADMINISTRACAO PUBLICA EM GERAL ...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,GRANDE,ADMINISTRAÇÃO PÚBLICA DIRETA - GOVERNO MUNICIPAL,----------,----------,"OUTRA, DE NATUREZA ESPECÍFICA OU MISTA",----------,LIQUIDADO
1,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03,706600.00,745030.36,RECURSOS ESTATUTÁRIOS - PRÓPRIOS ESTATUTÁRIOS,SEM CUSTO,0.0,0,0,NÃO REEMBOLSÁVEL,DIRETA,BNDES NÃO REEMBOLSÁVEL,FUNDO SOCIAL,NÃO,AREA DE GESTAO PUBLICA E SOCIOAMBIENTAL,COMERCIO E SERVICOS,SAÚDE E SERV SOCIAL,Q8700000,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
2,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03,75691.59,78500.00,RECURSOS ESTATUTÁRIOS - PRÓPRIOS ESTATUTÁRIOS,SEM CUSTO,0.0,0,0,NÃO REEMBOLSÁVEL,DIRETA,BNDES NÃO REEMBOLSÁVEL,FUNDO SOCIAL,NÃO,AREA DE GESTAO PUBLICA E SOCIOAMBIENTAL,COMERCIO E SERVICOS,SAÚDE E SERV SOCIAL,Q8700000,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
3,INSTITUTO DE DESENVOLVIMENTO SUSTENTAVEL DO BA...,02.275.306/0001-86,PROJETO DIREITO E CIDADANIA; PROJETO DEMONSTRA...,BA,ITUBERA,2917300,1247221,2002-01-03,603981.41,653834.69,RECURSOS ESTATUTÁRIOS - PRÓPRIOS ESTATUTÁRIOS,SEM CUSTO,0.0,0,0,NÃO REEMBOLSÁVEL,DIRETA,BNDES NÃO REEMBOLSÁVEL,FUNDO SOCIAL,NÃO,AREA DE GESTAO PUBLICA E SOCIOAMBIENTAL,COMERCIO E SERVICOS,SAÚDE E SERV SOCIAL,Q8700000,ATIV ATENCAO A SAUDE HUMANA INTEGR C/ASSIST SO...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,MICRO,PRIVADA,----------,----------,NÃO SE APLICA,----------,LIQUIDADO
4,ACEF S/A,46.722.831/0001-78,AQUISICAO DE EQUIPAMENTOS NACIONAIS E MOBILIAR...,SP,FRANCA,3516200,1255421,2002-01-09,340201.00,340189.32,RECURSOS LIVRES - FAT,TJLP,4.0,24,96,REEMBOLSÁVEL,INDIRETA,BNDES FINEM,PROGRAMA IES,NÃO,AREA DE GESTAO PUBLICA E SOCIOAMBIENTAL,COMERCIO E SERVICOS,EDUCAÇÃO,P8531700,EDUCACAO SUPERIOR - GRADUACAO ...,COMERCIO/SERVICOS,COMÉRCIO E SERVIÇOS,GRANDE,PRIVADA,BANCO BRADESCO S.A.,60746948000112,DEFINIDA PELO AGENTE FINANCEIRO,----------,LIQUIDADO


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19565 entries, 0 to 19564
Data columns (total 34 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   cliente                                     19565 non-null  object 
 1   cnpj                                        19565 non-null  object 
 2   descricao_do_projeto                        19565 non-null  object 
 3   uf                                          19565 non-null  object 
 4   municipio                                   19565 non-null  object 
 5   municipio_codigo                            19565 non-null  int64  
 6   numero_do_contrato                          19565 non-null  int64  
 7   data_da_contratacao                         19565 non-null  object 
 8   valor_contratado_reais                      19565 non-null  float64
 9   valor_desembolsado_reais                    19565 non-null  float64
 10  fonte_de_r

In [ ]:
df.to_parquet("dados/{arquivo}.parquet",engine='pyarrow',compression='gzip')